In [1]:
import pandas as pd
from pymongo import MongoClient

C:\Users\Tay Han\AppData\Local\Temp\ipykernel_13708\577788817.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
user_0 = pd.read_csv('user_0.csv')
user_1 = pd.read_csv('user_1.csv')
user_2 = pd.read_csv('user_2.csv')
user_3 = pd.read_csv('user_3.csv')
user_4 = pd.read_csv('user_4.csv')
user_5 = pd.read_csv('user_5.csv')
user_6 = pd.read_csv('user_6.csv')
user_7 = pd.read_csv('user_7.csv')
user_8 = pd.read_csv('user_8.csv')

In [3]:
print(len(user_0) + len(user_1))
print()

30077



In [4]:
#Stacking the CSVs together

users = pd.concat([user_0,
                user_1, 
                user_2,
                user_3,
                user_4, 
                user_5,
                user_6,
                user_7, 
                user_8])

In [5]:
#Aggregate all duplicate users within each sampling procedure
log = {col: 'mean' if users[col].dtype in ['float64', 'int64'] else 'first' for col in users.columns if col != 'source_user_id'}

users_result = users.groupby('source_user_id').agg(log).reset_index()


In [6]:
users_result["label"].value_counts()

label
0.0    59769
1.0    49656
Name: count, dtype: int64

In [8]:
#Concatenate all tweet relationships

graph_0 = pd.read_csv('graph_0.csv')
graph_1 = pd.read_csv('graph_1.csv')
graph_2 = pd.read_csv('graph_2.csv')
graph_3 = pd.read_csv('graph_3.csv')
graph_4 = pd.read_csv('graph_4.csv')
graph_5 = pd.read_csv('graph_5.csv')
graph_6 = pd.read_csv('graph_6.csv')
graph_7 = pd.read_csv('graph_7.csv')
graph_8 = pd.read_csv('graph_8.csv')

In [9]:
print(graph_0.dtypes)
print(graph_1.dtypes)
print(graph_2.dtypes)
print(graph_3.dtypes)
print(graph_4.dtypes)
print(graph_5.dtypes)
print(graph_6.dtypes)
print(graph_7.dtypes)
print(graph_8.dtypes)


source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int64
target_user_id     float64
relationship        object
tweet_embedding     object
dtype: object
source_user_id       int

In [11]:
#Stacking the tweet CSVs together

graph = pd.concat([graph_0,
                graph_1, 
                graph_2,
                graph_3,
                graph_4, 
                graph_5,
                graph_6,
                graph_7, 
                graph_8])

In [12]:
users_result.to_csv("users.csv", index = False)
graph.to_csv("graph.csv", index = False)

In [14]:
users_result['source_user_id'].value_counts()

source_user_id
22                     1
4440795449             1
4439992828             1
4439752995             1
4439364795             1
                      ..
266360210              1
266352448              1
266348259              1
266315980              1
1498176452084256772    1
Name: count, Length: 109425, dtype: int64

In [15]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder

# Create a mapping of user IDs to node indices >> change it up to index
user_ids = users_result["source_user_id"]
user_labels = users_result["label"]
user_to_idx = {user_id: idx for idx, user_id in enumerate(users_result["source_user_id"])}

# Create edge index and edge weight tensors
edge_list = []
edge_weight_dict = {}

for _, row in graph.iterrows():
    source_idx = user_to_idx[row['source_user_id']]
    target_idx = user_to_idx[row['target_user_id']]
    idx_pair = (source_idx, target_idx)
    if (idx_pair not in edge_list) :
        edge_list.append(idx_pair)
        edge_weight_dict[idx_pair] = 1
    else:
        edge_weight_dict[idx_pair] += 1

edge_weight_list = list(edge_weight_dict.values())

#force it to be contiguous
edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
edge_weight = torch.tensor(edge_weight_list, dtype=torch.float)

C:\Users\Tay Han\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyError: 7.325210585076204e+17